# Manipulador MK2: Este manipulador cuenta con 3 grados de libertad dados por 3 actuadores tipo servomotor.

<img src="https://raw.githubusercontent.com/uwulises/ME3250_tareas_laboratorios/refs/heads/main/mk2.png" width="500">

### Requerimientos

In [1]:
%%capture
# Install necessary libraries
!pip install ipywidgets
!pip install matplotlib

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# transformaciones

class Transformations:

    def translation_transform(self,x, y, z):

        return np.array([[1, 0, 0, x], [0, 1, 0, y], [0, 0, 1, z], [0, 0, 0, 1]])


    def rot_x(self, qx):

        return np.array([[1, 0, 0, 0], [0, np.cos(qx), -np.sin(qx), 0], [0, np.sin(qx), np.cos(qx), 0], [0, 0, 0, 1]])


    def rot_y(self, qy):

        return np.array([[np.cos(qy), 0, np.sin(qy), 0], [0, 1, 0, 0], [-np.sin(qy), 0, np.cos(qy), 0], [0, 0, 0, 1]])


    def rot_z(self, qz):

        return np.array([[np.cos(qz), -np.sin(qz), 0, 0], [np.sin(qz), np.cos(qz), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

In [4]:
class PoseGenerator:
    def __init__(self):
        self.fig=None
        self.fig3D = None
        self.ax=None
        self.ax3D=None

    def create_fig3D(self):
            self.fig3D = plt.figure(figsize=(10, 6))
            self.ax3D = self.fig3D.add_subplot(111, projection='3d')
            self.ax3D.axis('equal')
            self.ax3D.set_xlim([-210,210])
            self.ax3D.set_ylim([-210,210])
            self.ax3D.set_zlim([-210,210])
            return self.ax3D, self.fig3D

    def show_fig(self):
        plt.show()

    def draw_axes_tf(self, pose, name="", color="k",marker_length=50):
                self.ax3D.set_title("MLF MK2")
                origin_pose = np.transpose(pose)[3, 0:3]
                x_rot = np.linalg.multi_dot([pose, [1, 0, 0, 0]])
                y_rot = np.linalg.multi_dot([pose, [0, 1, 0, 0]])
                z_rot = np.linalg.multi_dot([pose, [0, 0, 1, 0]])
                self.ax3D.quiver(origin_pose[0], origin_pose[1], origin_pose[2], x_rot[0],
                        x_rot[1], x_rot[2], length=marker_length, normalize=True, color='r')
                self.ax3D.quiver(origin_pose[0], origin_pose[1], origin_pose[2], y_rot[0],
                        y_rot[1], y_rot[2], length=marker_length, normalize=True, color='g')
                self.ax3D.quiver(origin_pose[0], origin_pose[1], origin_pose[2], z_rot[0],
                        z_rot[1], z_rot[2], length=marker_length, normalize=True, color='b')
                self.ax3D.scatter(xs=[origin_pose[0]], ys=[origin_pose[1]],zs=[origin_pose[2]], marker='o', label=name)
                self.ax3D.legend(loc='upper right', bbox_to_anchor=(1.5, 1))


## Construcción de manipulador MK2 utilizando transformaciones (útil para entender la cinemática directa)

In [5]:
def MK2(q0=90,q1=90,q2=90, L1=135.0,L2=147.0, a1=100.0, eff_off_x=66.3,eff_off_z=0.0):
  #deg to rad
  q0 = q0*np.pi/180
  q1 = q1*np.pi/180
  q2 = q2*np.pi/180

  TF = Transformations()
  Origen = np.identity(4) #pose de origen
  #creamos las transformadas
  T_a1 = TF.translation_transform(0,0,a1)
  T_L1 = TF.translation_transform(0,0,L1)
  T_L2 = TF.translation_transform(0,0,-L2)
  T_tool = TF.translation_transform(eff_off_x,0,-eff_off_z)

  #creamos las rotaciones
  Rz_0 = TF.rot_z(np.pi/4-q0/2)
  Ry_1 = TF.rot_y(q1-np.pi/2)
  Rz_servo = TF.rot_z(np.pi) #compensacion de sentido del servo
  Ry_2 = TF.rot_y(q1+q2-np.pi/2)
  #compensar Ry_1 , Ry_2
  R_wrist = np.linalg.inv(np.linalg.multi_dot([Ry_1,Rz_servo,Ry_2]))
  #creamos las poses
  q0_pose = np.linalg.multi_dot([Origen,Rz_0])
  q1_pose = np.linalg.multi_dot([q0_pose,T_a1,Ry_1])
  q2_pose = np.linalg.multi_dot([q1_pose,T_L1,Rz_servo,Ry_2])
  wrist_pose = np.linalg.multi_dot([q2_pose,T_L2,R_wrist])
  tip_pose = np.linalg.multi_dot([wrist_pose,T_tool])

  M_o_n =  tip_pose
  M_o_n = np.round(M_o_n,2)

  #plot de poses
  PG = PoseGenerator()
  PG.create_fig3D()
  PG.draw_axes_tf(Origen,"Pose origen")
  PG.draw_axes_tf(q0_pose,"q0 Pose")
  PG.draw_axes_tf(q1_pose,"q1 Pose")
  PG.draw_axes_tf(q2_pose,"q2 Pose")
  PG.draw_axes_tf(wrist_pose,"Wrist Pose")
  PG.draw_axes_tf(M_o_n,"Tip Pose")
  PG.show_fig()
  print("Matriz resultante desde la base al efector: \n",M_o_n)
  print("\n")
  print("Coordenadas X,Y,Z:",np.round(np.transpose(M_o_n)[3,0:3],2))
  print("\n")
  return M_o_n

interact(MK2,q0=(0, 180, 1),q1=(0, 180, 1),q2=(0, 180, 1), eff_off_x=(0, 200, 1),eff_off_z=(0, 200, 1), L1=fixed(135.0), L2=fixed(147.0), a1=fixed(100.0))


interactive(children=(IntSlider(value=90, description='q0', max=180), IntSlider(value=90, description='q1', ma…

<function __main__.MK2(q0=90, q1=90, q2=90, L1=135.0, L2=147.0, a1=100.0, eff_off_x=66.3, eff_off_z=0.0)>

## Ahora con la cinemática inversa

### Solución analítica 3D

In [6]:
def inverse_kinematics(X=150,Y=0,Z=60, eff_off_x=66.3, eff_off_z=0):

    offset_z = 100
    zb = Z - offset_z + eff_off_z
    l1 = 135
    l2 = 147

    q0 = np.arctan2(Y,X)

    if np.abs(Y)<1e-2:
        xo = X-eff_off_x
    else:
        xo = np.sqrt((X-eff_off_x*np.cos(q0))**2 + (Y-eff_off_x*np.sin(q0))**2)

    q1 = np.pi - np.arctan2(zb,xo) - np.arccos((xo**2 + zb**2 + l1**2-l2**2)/(2*l1*np.sqrt(xo**2 + zb**2)))
    q2 = np.pi/2 - q1 + np.arccos((l1**2+l2**2-xo**2-zb**2)/(2*l1*l2))

    #to deg
    q0 = -2*np.round(np.rad2deg(q0),0)+90
    q1 = np.round(np.rad2deg(q1),0)
    q2 = np.round(np.rad2deg(q2),0)
    return np.array([q0,q1,q2])

### Solución numérica 2D con método Newton-Raphson

In [7]:

def mk2(x, x_pos, z_pos, L1=135.0, L2=147.0, d1=66.3,d2=0,a1=100.0):
  f1 = L1*np.cos(np.pi-x[0]) + L2*np.cos(x[1]-np.pi*0.5)+d1-x_pos
  f2 = a1 + L1*np.sin(np.pi-x[0]) + L2*np.sin(x[1]-np.pi*0.5)-d2-z_pos
  return np.array([f1,f2])

def Jmk2(x, L1=135.0,L2=147.0):
  J1 = [L1*np.sin(np.pi-x[0]),-L2*np.sin(x[1]-np.pi*0.5)]
  J2 = -L1*np.cos(np.pi-x[0]),L2*np.cos(x[1]-np.pi*0.5)
  return  np.array([J1,J2])

def NR_2D_inverse_kinematics(target_pos, init_guess=np.array([np.deg2rad(140),np.deg2rad(50)]),eff_x=66.3,eff_z=0):
  x = init_guess
  g = mk2(x,target_pos[0],target_pos[1],d1=eff_x,d2=eff_z)
  x= x-np.dot(np.linalg.inv(Jmk2(x)),g)
  new_g = mk2(x,target_pos[0],target_pos[1],d1=eff_x,d2=eff_z)
  it_num = 1
  while np.linalg.norm(g-new_g)>1e-10:
      g = new_g
      x= x-np.dot(np.linalg.inv(Jmk2(x)),g)
      new_g = mk2(x,target_pos[0],target_pos[1],d1=eff_x,d2=eff_z)
      it_num += 1
      if it_num > 1e3:
        print("No converge")
        break
  theta_1 = np.rad2deg(x[0])
  theta_2 = np.rad2deg(x[1])
  theta_1 = np.round(theta_1,0)
  theta_2 = np.round(theta_2,0)
  return np.array([theta_1,theta_2])


### Probemos las soluciones

In [8]:
target=np.array([240.0,90.0]) #target para X y Z

ang_analitica = inverse_kinematics(X=target[0],Z=target[1],eff_off_x=66,eff_off_z=24)
print("Solucion analitica: ", ang_analitica)
ang_numerica = NR_2D_inverse_kinematics(target,eff_x=66,eff_z=24)
print("Solución numérica: ", ang_numerica)

Solucion analitica:  [ 90. 120.  46.]
Solución numérica:  [120.  46.]
